# Documentation

## Comments and docstrings

In general:

- Try to write code that doesn't need comments.
- Either make comments useful or leave them out.
- Comments should be complete sentences.
- Use correct punctuation.
- Be brief.
- Only use string literals for docstrings.
- Use two spaces before an in-line comment.
- Put a space after the # symbol.
- Give scientific citations where appropriate.
- Don't comment out code you don't use, delete it.

In [1]:
def ignore_case(x):
    """
    This is a docstring. It's special.
    
    Args:
        x (str). The input arguments.
        
    Returns:
        str. What the function returns.
    """
    # This is just a normal comment.
    return x.lower()  # So is this.

In [2]:
help(ignore_case)

Help on function ignore_case in module __main__:

ignore_case(x)
    This is a docstring. It's special.
    
    Args:
        x (str). The input arguments.
        
    Returns:
        str. What the function returns.



In [3]:
print(ignore_case.__doc__)


    This is a docstring. It's special.
    
    Args:
        x (str). The input arguments.
        
    Returns:
        str. What the function returns.
    


In [4]:
# In Jupyter Notebook
ignore_case?

You'll find that all built-in functions and libraries, and everything in big 3rd party libraries like NumPy, has great documentation:

In [5]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



### Poor style

In [6]:
a = 9.81  # Gravitational acceleration.
b = 5     # Time in seconds.
c = (1/2) * a * (b**2)  # Calculate displacement.

This is better:

In [7]:
# Calculate displacement s given acceleration due to gravity and
# time, according to kinematic equation for constant acceleration.
g = 9.81
t = 5
s = (1/2) * g * (t**2)

This might be better still, depending on your opinion:

In [8]:
accelation_gravity = 9.81
time_in_s = 5
displacement_in_m = (1/2) * accelation_gravity * (time_in_s**2)

## A note about units

Units are a bit of a problem in scientific computing. There are several solutions, the best of which are probably `pint` and `astropy.units`. Here's `pint` in action:

In [15]:
import pint

ur = pint.UnitRegistry()

g = 9.81 * ur.m / (ur.s)**2
t = 5 * ur.s
s = (1/2) * g * (t**2)

s

<Quantity(122.625, 'meter')>

## Writing complete docs with `sphinx`

## `doctest`

Make your docstrings work for a living!

In [14]:
def quad(x, a=1, b=1, c=0):
    """
    Returns the quadratic function of x,
    a.x^2 + b.x + c
    where
    a = b = 1 and c = 0.
    
    Examples:
    >>> quad(10)
    110
    >>> quad(10, a=3, b=2, c=1)
    321
    """
    return a*x**2 + b*x + c

In [15]:
quad(10)

110

In [16]:
quad(10, a=3, b=2, c=1)

321

In [18]:
import doctest
doctest.run_docstring_examples(quad, globals(), verbose=True)

Finding tests in NoName
Trying:
    quad(10)
Expecting:
    110
ok
Trying:
    quad(10, a=3, b=2, c=1)
Expecting:
    321
ok


In [19]:
doctest.testmod()

TestResults(failed=0, attempted=2)

In [20]:
from IPython.core.magic import register_line_magic
import doctest

@register_line_magic
def testit(_):
    return doctest.testmod()

In [21]:
%testit

TestResults(failed=0, attempted=2)

In [22]:
from functools import wraps
import doctest

def test(func):
    @wraps(func)
    def f(*args, **kwargs):
        return func(*args, **kwargs)
    doctest.run_docstring_examples(func, globals())
    return f

In [24]:
@test
def quadd(x, a=1, b=1, c=0):
    """
    Returns the quadratic function of x,
    a.x^2 + b.x + c
    where
    a = b = 1 and c = 0.
    
    Examples:
    >>> quadd(10)
    110
    >>> quadd(10, a=3, b=2, c=1)
    321
    >>> quadd(0, c=5)
    4
    """
    return a*x**2 + b*x + c

**********************************************************************
File "__main__", line 14, in NoName
Failed example:
    quadd(0, c=5)
Expected:
    4
Got:
    5


In [26]:
quadd(0, c=5)

5

## Type hints

New in Python 3. Essentially a type of documentation. [Read about them.](https://docs.python.org/3/library/typing.html) [Read PEP484](https://www.python.org/dev/peps/pep-0484/).

You can check the internal consistency of types using [mypy](http://mypy-lang.org/index.html).

Python is **strongly typed** — you cannot add an `int` to a `str`. For example, `2 + "3"` throws a `TypeError`, whereas in JavaScript, which is weakly typed, it returns `"23"`. 

But Python is **dynamically typed**, so I can do `x = 5` and then, later, `x = "Hello"` — the type of `x` is dynamic, and depends only on the data I point it to. Similarly, I can pass ints, floats or strings into a function that multiplies things:

In [34]:
def double(n):
    return 2 * n

double('this')

'thisthis'

As you might imagine, sometimes this kind of flexibility can be the cause of bugs. 

The basic idea of type hints is to bridge the gap between dynamic typing (Python's usual mode, so to speak), and static typing (a popular feature of some other languages, such as Java or C).

You can annotate a variable assignment with the expected type of the variable, for example:

In [35]:
n: float = 3.14159

There's a similar signature for annotating functions, with some special syntax for annotating the return variable too:

In [36]:
def double(n: float) -> float:
    return 2 * n

double(2.1)

4.2

These are just annotations, however, there is no actual type checking. You can still do whatever you want.

In [37]:
double('this')

'thisthis'

You can, however, check the internal consistency of types using [mypy](http://mypy-lang.org/index.html).

The `typing` module helps make composite types (e.g. a list of floats), new types, etc.

In [38]:
from typing import List
Vector = List[float]

def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

# typechecks; a list of floats qualifies as a Vector.
new_vector = scale(2.0, [1.0, -4.2, 5.4])

In [39]:
new_vector

[2.0, -8.4, 10.8]

None of this changes the actual type of the variables:

In [40]:
type(new_vector)

list